In [65]:
import os
import re
import json
import base64
import tiktoken
import time
import pandas as pd
import openai
from tqdm.notebook import tqdm
from openai import OpenAI


In [66]:
# MODEL_LIMITS = {
#     "gpt-3.5-turbo-0125": 16_385,
#     "gpt-4-turbo-2024-04-09": 128_000,
#     "gpt-4o-2024-05-13": 128_000
# }

# # The cost per token for each model input.
# MODEL_COST_PER_INPUT = {
#     "gpt-3.5-turbo-0125": 0.0000005,
#     "gpt-4-turbo-2024-04-09": 0.00001,
#     "gpt-4o-2024-05-13": 0.000005
# }

# # The cost per token for each model output.
# MODEL_COST_PER_OUTPUT = {
#     "gpt-3.5-turbo-0125": 0.0000015,
#     "gpt-4-turbo-2024-04-09": 0.00003,
#     "gpt-4o-2024-05-13": 0.000015
# }



In [67]:
MODEL_LIMITS = {
    "gpt-3.5-turbo": 16385,
    "gpt-4": 128000
}

MODEL_COST_PER_INPUT = {
    "gpt-3.5-turbo": 0.0000005,
    "gpt-4": 0.00001
}

MODEL_COST_PER_OUTPUT = {
    "gpt-3.5-turbo": 0.0000015,
    "gpt-4": 0.00003
}


In [68]:
# If the question is a multi-choice question and you are unsure which one is correct, you must guess an option.  Please don't ask me any questions and give me the answer in the response.

def get_gpt_res(text, image, model):
    if image and model[:7] != "gpt-3.5":
        base64_image = encode_image(image)
        image_code = {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"}
        }
    
        response = client.chat.completions.create(
          model=model,
          messages=[
            {
              "role": "system",
              "content": [
                {
                  "type": "text",
                  "text": "You are a data analyst. I will give  you a background introduction and data analysis question. You must answer the question.  "
                }
              ]
            },
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": text
                },
                image_code
              ]
            }
          ],
          temperature=0,
          max_tokens=2256,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
        )
    else:
        response = client.chat.completions.create(
          model=model,
          messages=[
            {
              "role": "system",
              "content": [
                {
                  "type": "text",
                  "text": "You are a data analyst. I will give  you a background introduction and data analysis question. You must answer the question. "
                }
              ]
            },
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": text
                }
              ]
            }
          ],
          temperature=0,
          # max_tokens=2256,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
        )
    return response

In [69]:
samples = []
with open("./data.json", "r") as f:
    for line in f:
        samples.append(eval(line.strip()))


In [70]:
def gpt_tokenize(string: str, encoding) -> int:
    """Returns the number of tokens in a text string."""
    num_tokens = len(encoding.encode(string))
    return num_tokens

def find_jpg_files(directory):
    jpg_files = [file for file in os.listdir(directory) if file.lower().endswith('.jpg') or file.lower().endswith('.png')]
    return jpg_files if jpg_files else None

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


def find_excel_files(directory):
    jpg_files = [file for file in os.listdir(directory) if (file.lower().endswith('xlsx') or file.lower().endswith('xlsb') or file.lower().endswith('xlsm')) and not "answer" in file.lower()]
    return jpg_files if jpg_files else None

# def read_excel(file_path):
#     # 读取Excel文件中的所有sheet
#     xls = pd.ExcelFile(file_path)
#     sheets = {}
#     for sheet_name in xls.sheet_names:
#         sheets[sheet_name] = xls.parse(sheet_name)
#     return sheets
def read_excel(file_path):
    try:
        # Try with openpyxl (most common for .xlsx)
        xls = pd.ExcelFile(file_path, engine='openpyxl')
    except Exception as e1:
        try:
            # Try with pyxlsb (for .xlsb)
            xls = pd.ExcelFile(file_path, engine='pyxlsb')
        except Exception as e2:
            try:
                # Try with xlrd (for .xls)
                xls = pd.ExcelFile(file_path, engine='xlrd')
            except Exception as e3:
                print(f"\n Skipping file: {file_path}")
                print("Could not read the Excel file with any engine.")
                print("openpyxl error:", e1)
                print("pyxlsb error:", e2)
                print("xlrd error:", e3)
                return {}  # Return empty dict so processing continues

    sheets = {}
    for sheet_name in xls.sheet_names:
        try:
            sheets[sheet_name] = xls.parse(sheet_name)
        except Exception as e:
            print(f"⚠️ Skipped sheet '{sheet_name}' in {file_path}: {e}")
    return sheets


def dataframe_to_text(df):
    # 将DataFrame转换为文本
    text = df.to_string(index=False)
    return text

def combine_sheets_text(sheets):
    # 将所有sheet的文本内容组合起来
    combined_text = ""
    for sheet_name, df in sheets.items():
        sheet_text = dataframe_to_text(df)
        combined_text += f"Sheet name: {sheet_name}\n{sheet_text}\n\n"
    return combined_text

def read_txt(path):
    with open(path, "r",encoding="utf-8") as f:
        return f.read()

def truncate_text(text, max_tokens=128000):
    # 计算当前文本的token数
    tokens = text.split()
    if len(tokens) > max_tokens:
        # 截断文本以确保不超过最大token数
        text = ' '.join(tokens[-max_tokens:])
    return text

In [ ]:
client = OpenAI()

tokens4generation = 6000
# model = "gpt-3.5-turbo-0125"
model = "gpt-3.5-turbo"
data_path = "./data/"
total_cost = 0
encoding = tiktoken.encoding_for_model(model)
for id in tqdm(range(len(samples))):
    # print(sample)
    sample =samples[id]
    if len(sample["questions"]) > 0:
        start = sample["questions"][0]
        end = sample["questions"][-1]
        # print(start)
        # print(end)
        image = find_jpg_files(os.path.join(data_path, sample["id"]))
        if image:
            image = os.path.join(data_path, sample["id"], image[0])
        
        excel_content = ""
        excels = find_excel_files(os.path.join(data_path, sample["id"]))
        if excels:
            
            for excel in excels:
                excel_file_path = os.path.join(data_path,  sample["id"], excel)
                # print(excel_file_path)
                sheets = read_excel(excel_file_path)
                combined_text = combine_sheets_text(sheets)
                excel_content += f"The excel file {excel} is: " + combined_text

        introduction = read_txt(os.path.join(data_path, sample["id"], "introduction.txt"))
        questions = []
        for question_name in sample["questions"]:
            questions.append(read_txt(os.path.join(data_path, sample["id"], question_name+".txt")))
            
        # print(workbooks)
        
        text = ""
        if excel_content:
            text += f"The workbook is detailed as follows. {excel_content} \n"
        text += f"The introduction is detailed as follows. \n {introduction} \n"
        answers = []
        for question in questions:
            prompt = text +  f"The questions are detailed as follows. \n {question}"
        
            # print(len(encoding.encode(prompt)))
            cut_text = encoding.decode(encoding.encode(prompt)[tokens4generation-MODEL_LIMITS[model]:])
            # print(len(encoding.encode(prompt)))
            # print(prompt)
        # text = truncate_text(text, 20000)
            start = time.time()
            response = get_gpt_res(cut_text, image, model)
            cost = response.usage.completion_tokens * MODEL_COST_PER_OUTPUT[model] + response.usage.prompt_tokens * MODEL_COST_PER_INPUT[model]
            
            answers.append({"id": sample["id"], "model": response.model, "input": response.usage.prompt_tokens,
                            "output": response.usage.completion_tokens, "cost": cost, "time": time.time()-start, "response": response.choices[0].message.content})
            total_cost += cost
            print("Total cost: ", total_cost)
            # break
        save_path = os.path.join("./save_process", model)
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        with open(os.path.join(save_path, sample['id']+".json"), "w") as f:
            for answer in answers:
                json.dump(answer, f)
                f.write("\n")


  0%|          | 0/38 [00:00<?, ?it/s]

Total cost:  0.0055379999999999995
Total cost:  0.011087999999999999
Total cost:  0.016644
Total cost:  0.0222795
Total cost:  0.0278775
Total cost:  0.033546
Total cost:  0.039398999999999997
Total cost:  0.045301499999999995
Total cost:  0.050927999999999994
Total cost:  0.056613
Total cost:  0.062315999999999996
Total cost:  0.0679095
Total cost:  0.0735015
Total cost:  0.0736875
Total cost:  0.0738895
Total cost:  0.074019
Total cost:  0.0742595
Total cost:  0.074423
Total cost:  0.0745055
Total cost:  0.0747525
Total cost:  0.074922
Total cost:  0.0750335
Total cost:  0.0752395
Total cost:  0.075438
Total cost:  0.075643
Total cost:  0.07587100000000001
Total cost:  0.07609600000000001
Total cost:  0.07644300000000001
Total cost:  0.07657000000000001
Total cost:  0.07673900000000002
Total cost:  0.07690700000000002
Total cost:  0.07711800000000002
Total cost:  0.07720500000000002
Total cost:  0.07737300000000003
Total cost:  0.07755400000000003
Total cost:  0.07773500000000003
Tot

c:\Users\Shreni\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Total cost:  1.2224305000000006
Total cost:  1.2279985000000007
Total cost:  1.2337060000000006
Total cost:  1.2393445000000007
Total cost:  1.2448960000000007
Total cost:  1.2501655000000007
Total cost:  1.2560260000000008
Total cost:  1.2618400000000007
Total cost:  1.2673165000000008
Total cost:  1.2728455000000007
Total cost:  1.2782890000000007
Total cost:  1.2845410000000006
Total cost:  1.2899755000000006
Total cost:  1.2953965000000005
Total cost:  1.3007875000000004
Total cost:  1.3063720000000005
Total cost:  1.3116790000000005
Total cost:  1.3170730000000004
Total cost:  1.3284280000000004
Total cost:  1.3338865000000004
Total cost:  1.3393270000000004
Total cost:  1.3447540000000004
Total cost:  1.3502800000000004
Total cost:  1.3558180000000004
Total cost:  1.3613560000000005
Total cost:  1.3671235000000004
Total cost:  1.3726780000000003
Total cost:  1.3782655000000004
Total cost:  1.3837405000000003
Total cost:  1.3892365000000002
Total cost:  1.3951030000000002
Total co

c:\Users\Shreni\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Total cost:  1.4165479999999997
Total cost:  1.4184014999999996
Total cost:  1.4202759999999996
Total cost:  1.4221069999999996
Total cost:  1.4242419999999996
Total cost:  1.4249709999999995
Total cost:  1.4258454999999994
Total cost:  1.4268024999999995
Total cost:  1.4277959999999996
Total cost:  1.4289249999999996
Total cost:  1.4300074999999997
Total cost:  1.4307299999999996
Total cost:  1.4315824999999995
Total cost:  1.4323364999999995
Total cost:  1.4337484999999994
Total cost:  1.4344644999999994
Total cost:  1.4345339999999993
Total cost:  1.4346279999999993
Total cost:  1.4347169999999994
Total cost:  1.4350164999999993
Total cost:  1.4353179999999992
Total cost:  1.4356409999999993
Total cost:  1.4357349999999993
Total cost:  1.4358369999999994
Total cost:  1.4361834999999994
Total cost:  1.4362969999999995
Total cost:  1.4364034999999995
Total cost:  1.4366714999999994
Total cost:  1.4368629999999993
Total cost:  1.4370919999999994
Total cost:  1.4372344999999993
Total co